In [10]:
import mlflow
import mlflow.sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.model_selection import train_test_split,cross_val_predict,StratifiedKFold
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [11]:
tracking_uri = "http://ec2-16-170-247-92.eu-north-1.compute.amazonaws.com:5000" 

In [12]:
df  = pd.read_csv("data/preprocessing_df.csv")
df

,Unnamed: 0,text,category,word_count,num_chars,num_punctuation_chars
0,0,family mormon never tried explain still stare ...,1,39,259,0
1,1,buddhism much lot compatible christianity espe...,1,196,1268,0
2,2,seriously say thing first get complex explain ...,-1,86,459,0
3,3,learned want teach different focus goal wrappi...,0,29,167,0
4,4,benefit may want read living buddha living chr...,1,112,690,0
...,...,...,...,...,...,...
199697,162975,crore paid neerav modi recovered congress lead...,-1,18,107,0
199698,162976,dear rss terrorist payal gawar modi killing pl...,-1,36,248,0
199699,162977,cover interaction forum left,0,9,50,0
199700,162978,big project came india modi dream project happ...,0,13,77,0


In [13]:
df.text = df.text.fillna("")


In [14]:
mlflow.set_tracking_uri(tracking_uri)
mlflow.set_experiment("Exp2 : BoW vs TfIdf")


<Experiment: artifact_location='s3://mlflow-bucket-114/101431507168865979', creation_time=1751117239198, experiment_id='101431507168865979', last_update_time=1751117239198, lifecycle_stage='active', name='Exp2 : BoW vs TfIdf', tags={}>

In [15]:
def run_experiment(vectorizer_type,ngram_range,vectorizer_max_features,vectorizer_name):
    if vectorizer_type == "BoW":
        vectorizer = CountVectorizer(ngram_range=ngram_range,max_features=vectorizer_max_features)
    else:
        vectorizer = TfidfVectorizer(ngram_range=ngram_range,max_features=vectorizer_max_features)
        
    X = df.text
    y = df.category
    
    X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=114,stratify=y)
    
    X_train = vectorizer.fit_transform(X_train)
    X_test = vectorizer.transform(X_test)
    
    with mlflow.start_run() as run:
        mlflow.set_tag("mlflow.runName", f"{vectorizer_name}_{ngram_range}_RandomForest")
        mlflow.set_tag("experiment_type", "feature_engineering")
        mlflow.set_tag("model_type", "RandomForestClassifier")

        # Add a description
        mlflow.set_tag("description", f"RandomForest with {vectorizer_name}, ngram_range={ngram_range}, max_features={vectorizer_max_features}")

        # Log vectorizer parameters
        mlflow.log_param("vectorizer_type", vectorizer_type)
        mlflow.log_param("ngram_range", ngram_range)
        mlflow.log_param("vectorizer_max_features", vectorizer_max_features)

        # Log Random Forest parameters
        n_estimators = 200
        max_depth = 15

        mlflow.log_param("n_estimators", n_estimators)
        mlflow.log_param("max_depth", max_depth)

        # Initialize and train the model
        model = RandomForestClassifier(n_estimators=n_estimators,class_weight="balanced" ,max_depth=max_depth, random_state=42)
        model.fit(X_train, y_train)

        # Step 5: Make predictions and log metrics
        y_pred = model.predict(X_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log confusion matrix
        conf_matrix = confusion_matrix(y_test, y_pred)
        plt.figure(figsize=(8, 6))
        sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.title(f"Confusion Matrix: {vectorizer_name}, {ngram_range}")
        plt.savefig("confusion_matrix.png")
        mlflow.log_artifact("confusion_matrix.png")
        plt.close()

        # Log the model
        mlflow.sklearn.log_model(model, f"random_forest_model_{vectorizer_name}_{ngram_range}")

ngram_ranges = [(1,1),(1,2),(1,3)]
max_features = 5000

for ngram_range in ngram_ranges:
    run_experiment("BoW",ngram_range,max_features,vectorizer_name="BoW")
    run_experiment("TF-IDF",ngram_range,max_features,vectorizer_name="TF-IDF")

2025/06/28 16:54:51 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/06/28 16:54:58 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run BoW_(1, 1)_RandomForest at: http://ec2-16-170-247-92.eu-north-1.compute.amazonaws.com:5000/#/experiments/101431507168865979/runs/a3ed2e68a9714763949ea4d6c7e0e38d
🧪 View experiment at: http://ec2-16-170-247-92.eu-north-1.compute.amazonaws.com:5000/#/experiments/101431507168865979


2025/06/28 16:55:34 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/06/28 16:55:40 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run TF-IDF_(1, 1)_RandomForest at: http://ec2-16-170-247-92.eu-north-1.compute.amazonaws.com:5000/#/experiments/101431507168865979/runs/1e33a999417840b39a496986d25e98ac
🧪 View experiment at: http://ec2-16-170-247-92.eu-north-1.compute.amazonaws.com:5000/#/experiments/101431507168865979


2025/06/28 16:56:15 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/06/28 16:56:21 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


...

🏃 View run TF-IDF_(1, 2)_RandomForest at: http://ec2-16-170-247-92.eu-north-1.compute.amazonaws.com:5000/#/experiments/101431507168865979/runs/6628ff1cf0ba49d6afafd80070080d95
🧪 View experiment at: http://ec2-16-170-247-92.eu-north-1.compute.amazonaws.com:5000/#/experiments/101431507168865979


2025/06/28 16:57:52 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/06/28 16:57:58 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run BoW_(1, 3)_RandomForest at: http://ec2-16-170-247-92.eu-north-1.compute.amazonaws.com:5000/#/experiments/101431507168865979/runs/05436dd8de674b04ba6e933233af066d
🧪 View experiment at: http://ec2-16-170-247-92.eu-north-1.compute.amazonaws.com:5000/#/experiments/101431507168865979


2025/06/28 16:58:43 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/06/28 16:58:49 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run TF-IDF_(1, 3)_RandomForest at: http://ec2-16-170-247-92.eu-north-1.compute.amazonaws.com:5000/#/experiments/101431507168865979/runs/9064bfda1ea24f1191cf437c68c2f2b4
🧪 View experiment at: http://ec2-16-170-247-92.eu-north-1.compute.amazonaws.com:5000/#/experiments/101431507168865979
